In [1]:
:opt no-lint

# Simply-typed $\lambda$-calculus (STLC)

단순타입 람다계산법(Simply-typed $\lambda$-calculus 또는 줄여서 STLC)

$\tau \in \textit{Ty}
      ::= \tau_1 \to \tau_2
    ~\mid~ \textit{Int}
$

$e \in \textit{Expr}
   ::=~ x ~\mid~ \lambda x.e ~\mid~ e_1~e_2 ~\mid~ \\\qquad\qquad
 ~\mid~ n ~\mid~ e_1 + e_2 ~\mid~ \texttt{if}~e~\texttt{then}~e_1~\texttt{else}~e_0
$

In [2]:
data Ty = Ty :-> Ty | INT | TV Nm deriving (Eq,Show)
infixr 9 :->

data Expr = Var Nm  | Lam Nm Expr | App Expr Expr
          | Val Int | Add Expr Expr | If Expr Expr Expr
        deriving Show

type Nm = String

In [3]:
ppTy (t1 :-> t2) = ppty t1 ++ "->" ++ ppTy t2
ppTy INT = "int"
ppTy (TV x) = x

ppty t@(_ :-> _) = paren (ppTy t)
ppty t           = ppTy t

-- 람다식을 보기좋게 문자열로 변환해주는 함수
ppTm (Var x) = x
ppTm (Lam x e) = "\\"++x++" -> " ++ ppTm e
ppTm (App e1 e2) = pp1 e1 ++ " " ++ pp2 e2
ppTm (Val n) = show n
ppTm (Add e1 e2) = ppp e1 ++ " + " ++ ppp e2
ppTm (If e e1 e0) = "if "++pp2 e++" then "++pp2 e1++" else "++pp2 e0 

pp1 e@(Lam{}) = paren (ppTm e)
pp1 e@(Add{}) = paren (ppTm e)
pp1 e@(If{})  = paren (ppTm e)
pp1 e         = ppTm e

pp2 e@(Var{}) = ppTm e
pp2 e@(Val{}) = ppTm e
pp2 e         = paren (ppTm e)

ppp e@(Var{}) = ppTm e
ppp e@(Val{}) = ppTm e
ppp e@(Add{}) = ppTm e
ppp e@(App{}) = ppTm e
ppp e         = paren (ppTm e)

paren s = "(" ++ s ++ ")"
brack s = "[" ++ s ++ "]"
latex s = "$" ++ s ++ "$"

texTy (t1 :-> t2) = texty t1 ++ "\\!\\to\\!" ++ texTy t2
texTy INT = "\\texttt{int}"

texty t@(_ :-> _) = paren (texTy t)
texty t           = texTy t

-- 람다식을 보기좋게 TeX 코드로 변환해주는 함수
texTm (Var x) = x
texTm (Lam x e) = "\\lambda " ++ x ++ "." ++ texTm e
texTm (App e1 e2) = tex1 e1 ++ "~" ++ tex2 e2
texTm (Val n) = show n
texTm (Add e1 e2) = texp e1 ++ "+" ++ texp e2
texTm (If e e1 e0) = "\\texttt{if}~"++tex2 e++"~\\texttt{then}~"++tex2 e1++"~\\texttt{else}~"++tex2 e0

tex1 e@(Lam{}) = paren (texTm e)
tex1 e@(Add{}) = paren (texTm e)
tex1 e@(If{}) = paren (texTm e)
tex1 t         = texTm t

tex2 s@(Var{}) = texTm s
tex2 s@(Val{}) = texTm s
tex2 s         = paren (texTm s)

texp s@(Var{}) = texTm s
texp s@(Val{}) = texTm s
texp s@(Add{}) = texTm s
texp s@(App{}) = texTm s
texp s         = paren (texTm s)

In [4]:
type TEnv = [ (Nm, Ty) ]
type Eqns = [ (Ty, Ty) ]

tyinf :: TEnv -> Expr -> [Nm] -> (Ty,Eqns)
tyinf tenv (Var x) _ = (lookup' x tenv, [])
tyinf tenv (Lam x e) (v:vs) = (t :-> t2, eqns2)
  where
    t = TV v
    (t2,eqns2) = tyinf ((x,t):tenv) e vs
tyinf tenv (App e1 e2) (v:vs) = (t, (t1,t2:->t):eqns1++eqns2)
  where
    t = TV v
    (t1,eqns1) = tyinf tenv e1 vs1
    (t2,eqns2) = tyinf tenv e2 vs2
    (vs1,vs2) = split2 vs
tyinf tenv (Val _) _ = (INT, [])
tyinf tenv (Add e1 e2) vs = (INT, (t1,INT):(t2,INT):eqns1++eqns2)
  where
    (t1,eqns1) = tyinf tenv e1 vs1
    (t2,eqns2) = tyinf tenv e2 vs2
    (vs1,vs2) = split2 vs
tyinf tenv (If e e1 e0) vs = (t1, (t,INT):(t1,t0):eqns++eqns1++eqns2)
  where
    (t,eqns) = tyinf tenv e vs'
    (t1,eqns1) = tyinf tenv e1 vs1
    (t0,eqns2) = tyinf tenv e0 vs0
    (vs',vs1,vs0) = split3 vs


lookup' x env = case lookup x env of
                  Nothing -> error (x ++ " not defined")
                  Just v  -> v

split2 (x:y:zs) = (x:xs,y:ys) where (xs,ys) = split2 zs
split2 [x] = ([x],[])
split2 [] = ([],[])

split3 (x:y:z:ws) = (x:xs,y:ys,z:zs) where (xs,ys,zs) = split3 ws
split3 [x,y] = ([x],[y],[])
split3 [x] = ([x],[],[])
split3 [] = ([],[],[])


tvars = [x:show i | i<-[0..], x<-['A'..'Z']]

In [5]:
import Data.List (intersperse)
-- :-set_prolog_flag(occurs_check,true).
ppTyEqns (t,eqns) = ppEqns eqns ++", write("++ppty t++")."
ppEqns eqns = concat . intersperse "," $ map (paren . ppEqn) eqns
ppEqn (t1,t2) = ppty t1++"="++ppty t2

In [6]:
import IHaskell.Display

add1 = Lam "x" $ Add (Val 1) (Var "x")
idTm = Lam "x" $ Var "x"

html . latex $ texTm add1
putStr . ppTyEqns $ tyinf [] add1 tvars

$\lambda x.1+x$

(int=int),(A0=int), write((A0->int)).

In [7]:
html . latex $ texTm (App add1 (Val 3))
putStr . ppTyEqns $ tyinf [] (App add1 (Val 3)) tvars

$(\lambda x.1+x)~3$

((B0->int)=(int->A0)),(int=int),(B0=int), write(A0).

In [8]:
html . latex $ texTm (App (Val 3) add1)
putStr . ppTyEqns $ tyinf [] (App (Val 3) add1) tvars

$3~(\lambda x.1+x)$

(int=((C0->int)->A0)),(int=int),(C0=int), write(A0).

In [9]:
html . latex $ texTm (App add1 idTm)
putStr . ppTyEqns $ tyinf [] (App add1 idTm) tvars

$(\lambda x.1+x)~(\lambda x.x)$

((B0->int)=((C0->C0)->A0)),(int=int),(B0=int), write(A0).

In [10]:
html . latex $ texTm (App idTm add1)
putStr . ppTyEqns $ tyinf [] (App idTm add1) tvars

$(\lambda x.x)~(\lambda x.1+x)$

((B0->B0)=((C0->int)->A0)),(int=int),(C0=int), write(A0).

In [11]:
html . latex $ texTm (App idTm idTm)
putStr . ppTyEqns $ tyinf [] (App idTm idTm) tvars

$(\lambda x.x)~(\lambda x.x)$

((B0->B0)=((C0->C0)->A0)), write(A0).